## The goal for this notebook is to create an autoencoder to sort out bad segmentations. Some of the 4000 images had odd artifacts. There are a few ways to filter out these poor segmentations. Here, we will try to use an autoencoder.

In [ ]:
# largely from : https://www.pyimagesearch.com/2020/03/02/anomaly-detection-with-keras-tensorflow-and-deep-learning/

In [ ]:
# import the necessary packages
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import tensorflow as tf
#tf.test.is_gpu_available()
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
        
import numpy as np
class ConvAutoencoder:
	@staticmethod
	def build(width, height, depth, filters=(32, 64), latentDim=16):
		# initialize the input shape to be "channels last" along with
		# the channels dimension itself
		# channels dimension itself
		inputShape = (height, width, depth)
		chanDim = -1
		# define the input to the encoder
		inputs = Input(shape=inputShape)
		x = inputs
		# loop over the number of filters
		for f in filters:
			# apply a CONV => RELU => BN operation
			x = Conv2D(f, (3, 3), strides=2, padding="same")(x)
			x = LeakyReLU(alpha=0.2)(x)
			x = BatchNormalization(axis=chanDim)(x)
		# flatten the network and then construct our latent vector
		volumeSize = K.int_shape(x)
		x = Flatten()(x)
		latent = Dense(latentDim)(x)
		# build the encoder model
		encoder = Model(inputs, latent, name="encoder")
		# start building the decoder model which will accept the
		# output of the encoder as its inputs
		latentInputs = Input(shape=(latentDim,))
		x = Dense(np.prod(volumeSize[1:]))(latentInputs)
		x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)
		# loop over our number of filters again, but this time in
		# reverse order
		for f in filters[::-1]:
			# apply a CONV_TRANSPOSE => RELU => BN operation
			x = Conv2DTranspose(f, (3, 3), strides=2,
				padding="same")(x)
			x = LeakyReLU(alpha=0.2)(x)
			x = BatchNormalization(axis=chanDim)(x)
		# apply a single CONV_TRANSPOSE layer used to recover the
		# original depth of the image
		x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
		outputs = Activation("sigmoid")(x)
		# build the decoder model
		decoder = Model(latentInputs, outputs, name="decoder")
		# our autoencoder is the encoder + decoder
		autoencoder = Model(inputs, decoder(encoder(inputs)),
			name="autoencoder")
		# return a 3-tuple of the encoder, decoder, and autoencoder
		return (encoder, decoder, autoencoder)

In [2]:
# set the matplotlib backend so figures can be saved in the background
import matplotlib
matplotlib.use("Agg")
# import the necessary packages
#from pyimagesearch.convautoencoder import ConvAutoencoder
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import argparse
import random
import pickle
import cv2

In [3]:
EPOCHS = 20
INIT_LR = 1e-3
BS = 32

(encoder, decoder, autoencoder) = ConvAutoencoder.build(28, 28, 1)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
autoencoder.compile(loss="mse", optimizer=opt)

In [4]:
((trainX, trainY), (testX, testY)) = mnist.load_data()

In [5]:
#rain the convolutional autoencoder
H = autoencoder.fit(
	trainX, trainX,
	validation_data=(testX, testX),
	epochs=EPOCHS,
	batch_size=BS)
# use the convolutional autoencoder to make predictions on the
# testing images, construct the visualization, and then save it
# to disk
print("[INFO] making predictions...")
decoded = autoencoder.predict(testX)
vis = visualize_predictions(decoded, testX)
cv2.imwrite(args["vis"], vis)

Epoch 1/20
1875/1875 [==============================] - 6s 3ms/step - loss: 7216.8218 - val_loss: 7342.8857
Epoch 2/20
1316/1875 [====================>.........] - ETA: 1s - loss: 7214.9233

KeyboardInterrupt: 

In [6]:
np.append()

(60000, 28, 28)

In [34]:
trainX.shape

(60000, 28, 28)

In [114]:
#loop through binary mask images
import os
binary_dir = '/tf/beetleData/images/binaryMask/'
count = 0
for i in os.listdir(binary_dir):
    if 'jpg' in i:
        if count == 0:        
            img_in = cv2.imread(binary_dir+i)
            print(img_in.shape)
            single_channels = img_in[:,:,0]
            single_channels = cv2.resize(single_channels,(100,100))
            single_channels = np.expand_dims(single_channels,axis=0)

            print(single_channel.shape)
            count+=1
        else:
            img_in = cv2.imread(binary_dir+i)
            #print(img_in.shape)
            single_channel = img_in[:,:,0]
            single_channel = cv2.resize(single_channel,(100,100))
            single_channel = np.expand_dims(single_channel,axis=0)
            #print('single channel ' + str(single_channel.shape))
            #print('channels ' + str(single_channels.shape))
            single_channels = np.concatenate((single_channels,single_channel),axis=0)
            count+=1
            if count % 50 == 0:
                print(count)
    

(670, 450, 3)
(1, 100, 100)
50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100
1150
1200
1250
1300
1350
1400
1450
1500
1550
1600
1650
1700
1750
1800
1850
1900
1950
2000
2050
2100
2150
2200
2250
2300
2350
2400
2450
2500
2550
2600
2650
2700
2750
2800
2850
2900
2950
3000
3050
3100
3150
3200
3250
3300
3350
3400
3450
3500
3550
3600
3650
3700
3750
3800
3850
3900
3950
4000
4050


In [115]:
single_channels.shape
# Now we have our MNIST-like data structure. Let's try to use it to train the autoencoder

(4074, 100, 100)

In [132]:
mytrainX, mytestX = train_test_split(single_channels)

In [133]:
trainX.shape

(60000, 28, 28)

In [136]:
mytrainX.shape

(3055, 100, 100)

In [127]:
testX.shape

(1019, 100, 100)

In [144]:
mytrainX[0,:,:][1:50,1:50]

array([[255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       [255, 255, 255, ..., 255, 255, 255],
       ...,
       [255, 255, 255, ...,   0,   0,   0],
       [255, 255, 255, ...,   0,   0,   0],
       [255, 255, 255, ...,   0,   0,   0]], dtype=uint8)

In [143]:
trainX[0,:,:][1:50,1:50]

array([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
          0],
       [  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,
         18,  18, 126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,
          0],
       [  0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 170, 253,
        253, 253, 253, 253, 225, 172, 253, 242, 195,  64,   0,   0,   0,
          0],
       [  0,   0,   0,   0,   0,   0,  4

In [149]:
class ConvAutoencoder:
	@staticmethod
	def build(width, height, depth, filters=(32, 64), latentDim=16):
		# initialize the input shape to be "channels last" along with
		# the channels dimension itself
		# channels dimension itself
		inputShape = (height, width, depth)
		chanDim = -1
		# define the input to the encoder
		inputs = Input(shape=inputShape)
		x = inputs
		# loop over the number of filters
		for f in filters:
			# apply a CONV => RELU => BN operation
			x = Conv2D(f, (3, 3), strides=2, padding="same")(x)
			x = LeakyReLU(alpha=0.2)(x)
			x = BatchNormalization(axis=chanDim)(x)
		# flatten the network and then construct our latent vector
		volumeSize = K.int_shape(x)
		x = Flatten()(x)
		latent = Dense(latentDim)(x)
		# build the encoder model
		encoder = Model(inputs, latent, name="encoder")
		# start building the decoder model which will accept the
		# output of the encoder as its inputs
		latentInputs = Input(shape=(latentDim,))
		x = Dense(np.prod(volumeSize[1:]))(latentInputs)
		x = Reshape((volumeSize[1], volumeSize[2], volumeSize[3]))(x)
		# loop over our number of filters again, but this time in
		# reverse order
		for f in filters[::-1]:
			# apply a CONV_TRANSPOSE => RELU => BN operation
			x = Conv2DTranspose(f, (3, 3), strides=2,
				padding="same")(x)
			x = LeakyReLU(alpha=0.2)(x)
			x = BatchNormalization(axis=chanDim)(x)
		# apply a single CONV_TRANSPOSE layer used to recover the
		# original depth of the image
		x = Conv2DTranspose(depth, (3, 3), padding="same")(x)
		outputs = Activation("sigmoid")(x)
		# build the decoder model
		decoder = Model(latentInputs, outputs, name="decoder")
		# our autoencoder is the encoder + decoder
		autoencoder = Model(inputs, decoder(encoder(inputs)),
			name="autoencoder")
		# return a 3-tuple of the encoder, decoder, and autoencoder
		return (encoder, decoder, autoencoder)

In [156]:
EPOCHS = 2
INIT_LR = 1e-3
BS = 32

(encoder, decoder, autoencoder) = ConvAutoencoder.build(100, 100, 1)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
autoencoder.compile(loss="mae", optimizer=opt)

In [158]:
#rain the convolutional autoencoder
H = autoencoder.fit(
	mytrainX, mytrainX,
	validation_data=(mytestX, mytestX),
	epochs=5,
	batch_size=2,
    verbose=1)
# use the convolutional autoencoder to make predictions on the
# testing images, construct the visualization, and then save it
# to disk
# print("[INFO] making predictions...")
# decoded = autoencoder.predict(testX)
# vis = visualize_predictions(decoded, testX)
# cv2.imwrite(args["vis"], vis)

Epoch 1/5
1528/1528 [==============================] - 6s 4ms/step - loss: 194.1493 - val_loss: 194.4836
Epoch 2/5
1528/1528 [==============================] - 6s 4ms/step - loss: 194.1358 - val_loss: 194.4814
Epoch 3/5
1528/1528 [==============================] - 6s 4ms/step - loss: 194.1330 - val_loss: 194.4788
Epoch 4/5
1528/1528 [==============================] - 6s 4ms/step - loss: 194.1314 - val_loss: 194.4778
Epoch 5/5
1528/1528 [==============================] - 6s 4ms/step - loss: 194.1304 - val_loss: 194.4769
